In [1]:
import threading
from datetime import datetime
from datetime import timedelta
from datetime import timezone as tzone
import time


In [2]:
class Scheduler:
    """
    A class to represent a schedule for jobs with start and end dates.
    
    Attributes:
    -----------
    threading : boolean 
        Whether or not to use threading while running the jobs 
    
    Methods:
    --------
    job():
        Method to create and configure a job
    run_all():
        Runs all jobs with or without threading
    """

    def __init__(self, threading = False, start_date = None, time_zone=None):
        
        # ensuring that we get appropriate type of parameters for Scheduler 
        if not isinstance(time_zone, int) and time_zone is not None:
            raise ValueError("Timezone must be an integer")

        if not isinstance(start_date, str) and start_date is not None:
            raise ValueError("Date must be an str format %Y-%m-%d %H:%M")

        self.threads = threading   #threading 
        self.scheduler_timezone = tzone(timedelta(hours=time_zone)) if time_zone  else datetime.now().astimezone().tzinfo # timezone of scheduler
        self.scheduler_startdate = (datetime.strptime(start_date, '%Y-%m-%d %H:%M') if start_date else datetime.now().replace(second=0, microsecond=0)).astimezone(self.scheduler_timezone) # job running start date passed to scheduler


        # validating that passed startdate isn't more than current time
        if  self.scheduler_startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Start date cannot be less than the current date")

        self.jobs = []  # empty list for adding jobs later


    def job(self, start_date=None, time_zone=None):

        """
        Creates and configures a job with a specified start date and time zone.
        
        Args:
        -----
        startdate : str, optional
            The start date of the job in the format '%Y-%m-%d %H:%M'. If not provided, the scheduler's default start date is used.
        
        timezone : int, optional
            The timezone offset in hours. If not provided, the scheduler's default timezone is used.
        
        Raises:
        -------
        ValueError
            If the timezone is not an integer or if the start date is not in the correct string format.
            If the job start date is set to a time earlier than the current date and time.
        
        Returns:
        --------
        JobWrapper
            A JobWrapper object that allows further configuration of the job.
        """
        # ensuring that we get appropriate type of parameters for job
        if not isinstance(time_zone, int) and time_zone is not None:
            raise ValueError("Timezone must be an integer")

        if not isinstance(start_date, str) and start_date is not None:
            raise ValueError("Date must be an str fromat %Y-%m-%d %H:%M")

        job_timezone = tzone(timedelta(hours=time_zone)) if time_zone else  self.scheduler_timezone # timezone of job
        job_startdate = datetime.strptime(start_date, '%Y-%m-%d %H:%M').astimezone(job_timezone) if start_date else self.scheduler_startdate # job running start date passed to job

        # ensuring that start date isn't in the past
        if  job_startdate.strftime('%Y-%m-%d %H:%M') < datetime.now().strftime('%Y-%m-%d %H:%M'):
            raise ValueError("Job start date cannot be less than the current date")

        # creating the job configuration as a dictionary with default values for job parameters
        job = {
            'startdate': job_startdate,
            'time_zone': job_timezone,
            'end_date': None,
            'func': None,
            'name': None,
            'unit': None,
            'interval': None,
            'next_run': None,
            'repeats': None,  # New attribute for repeats
            'repeat_count': 0,  # Counter for completed runs
            'args': (),
            'kwargs': {}
        }


        class JobWrapper:
            """
            A wrapper class to represent an individual job within the scheduler.
            
            Attributes:
            -----------
            job : dict
                Dictionary containing configuration parameters of the job.
            
            Methods:
            --------
            second(every=1):
                Sets the job to run every specified number of seconds. (code runs in every 1 second as default)
            minute(every=1):
                Sets the job to run every specified number of minutes. (code runs in every 1 minute as default)
            hour(every=1):
                Sets the job to run every specified number of hours. (code runs in every 1 hour as default)
            day(every=1, hour=None):
                Sets the job to run every specified number of days at an optional specific time. (code runs in every 1 day as default)
            week(every=1, week_day=None, hour=None):
                Sets the job to run every specified number of weeks on specified weekdays and time. (code runs in every 1 week as default)
            do(func, name):
                Configures the function to run with the job and assigns a name to the job.
            until(end date):
                Sets the end date for the job.
            repeat(times):
                Sets the number of times the job should repeat.
            calculate_next_run(current_time):
                Calculates the next run time of the job based on its schedule.
            run(*args, **kwargs):
                Executes the job at its scheduled times with the provided arguments.
            """
            
            def __init__(self, job):
                self.job = job  # passes the job that should get executed
                
                
            def second(self, every=1):
                """
                Sets the job to run every `every` seconds and defines unit as a second
                
                Args:
                -----
                every : Sets the job to run every specified number of seconds.
                """
                
                self.job['unit'] = 'second'  # time unit for the job
                self.job['interval'] = every # interval 
                return self

            def minute(self, every=1):
                """Set the job to run every `every` minutes."""
                self.job['unit'] = 'minute'
                self.job['interval'] = every
                return self

            def hour(self, every=1):
                """Set the job to run every `every` hours."""
                self.job['unit'] = 'hour'
                self.job['interval'] = every
                return self

            def day(self, every=1, hour=None):
                """Set the job to run every `every` days, optionally at a specific time."""
                self.job['unit'] = 'day'
                self.job['interval'] = every
                self.job['at'] = hour
                if hour:
                    # Parse the time in HH:MM format and update the next run time accordingly
                    hour, minute = map(int, hour.split(':'))
                    today = datetime.now(self.job['time_zone']).replace(hour=hour, minute=minute, second=0, microsecond=0)
                    # If the specified time has already passed today, set it for the next day
                    if today <= datetime.now(self.job['time_zone']):
                        today += timedelta(days=1)
                    self.job['next_run'] = today
                return self
            
            def week(self, every=1, week_day=None, hour=None):
                """
                Set the job to run every `every` weeks on specific weekdays at a specific time.
                If `week_day` is None, the job will every week. 
                
                Parameters:
                -----------
                every : int
                    Interval in weeks.
                week_day : int, list of int, or None
                    Single integer or list of weekdays where 0 = Monday, ..., 6 = Sunday. If None, run every day.
                hour : str
                    Time in HH:MM format at which the job should run.
                """
                if  week_day and  (type(week_day) == int or not type(week_day)==list):
                    print('in int or liat error')
                    raise ValueError("Weekday must be an integer or a list of integers")
                elif week_day and type(week_day) == list:
                    if isinstance(week_day, list) and not all(isinstance(i, int) for i in week_day):
                        print('in only list error')
                        raise ValueError('List of Weekdays must contain integers only')

                    


                self.job['unit'] = 'week'
                self.job['interval'] = every

                # making it possible to take integers and lists as week day
                self.job['week_day'] = [week_day] if isinstance(week_day, int) else week_day
                # defining at what hour of the day code should run
                self.job['at'] = hour
                return self
                                


            def do(self, func, name):
                self.job['func'] = func
                self.job['name'] = name
                # print('indo')
                return self


            def until(self, end_date):
                end_date = datetime.strptime(end_date, '%Y-%m-%d %H:%M')
                end_date = end_date.astimezone(self.job['time_zone'])
                if end_date < self.job['startdate']:
                    raise ValueError('Startdate must be earlier than End date')
                self.job['end_date'] = end_date
                print('inuntil')
                return self

            def repeat(self, times):
                """Set the number of times the job should repeat."""
                self.job['repeats'] = times
                print(f'inrepeat for {self.job["name"]}')
                return self

            def calculate_next_run(self, current_time):
                # Truncate the current_time to remove milliseconds
                current_time = current_time.replace(microsecond=0)

                if self.job['unit'] == 'second':
                    self.job['next_run'] = current_time + timedelta(seconds=self.job['interval'])
                    print(f'this is interval for second {self.job["interval"]}')
                elif self.job['unit'] == 'minute':
                    current_time = current_time.replace(second=0)
                    self.job['next_run'] = current_time + timedelta(minutes=self.job['interval'])
                elif self.job['unit'] == 'hour':
                    current_time = current_time.replace(minute=0, second=0)
                    self.job['next_run'] = current_time + timedelta(hours=self.job['interval'])
                elif self.job['unit'] == 'day':
                    # Handling the 'at' parameter for the day interval
                    if 'at' in self.job and self.job['at']:
                        # Parse the 'at' time from the job settings
                        hour, minute = map(int, self.job['at'].split(':'))
                        next_run = current_time.replace(hour=hour, minute=minute, second=0, microsecond=0)

                        # Ensure next_run is set on the correct day interval
                        if next_run <= current_time:
                            next_run += timedelta(days=self.job['interval'])
                        self.job['next_run'] = next_run
                    else:
                        # Default behavior without 'at' attribute
                        current_time = current_time.replace(hour=0, minute=0, second=0)
                        self.job['next_run'] = current_time + timedelta(days=self.job['interval'])
                elif self.job['unit'] == 'week':
                    # Set up the initial next run time based on the current time and interval
                    next_run = current_time
                    interval_weeks = self.job['interval']
                    # Default to current weekday if no week_day is specified
                    weekdays = self.job['week_day'] if self.job['week_day'] is not None else [current_time.weekday()]
            
                    # If no specific hour is provided, use the current hour
                    if not self.job.get('at'):
                        self.job['at'] = current_time.strftime('%H:%M')
            
                    hour, minute = map(int, self.job['at'].split(':'))
                    next_run = next_run.replace(hour=hour, minute=minute, second=0, microsecond=0)
            
                    # Find the nearest valid weekday
                    days_ahead = [(day - next_run.weekday()) % 7 for day in weekdays]
                    next_run += timedelta(days=min(days_ahead))
            
                    # If the next run is still in the past, adjust by adding the weekly interval
                    if next_run <= current_time:
                        next_run += timedelta(weeks=interval_weeks)
                    print(f'THISISWEEK{next_run}')
            
                    self.job['next_run'] = next_run
                else:
                    raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
                print(f"calc next run: {self.job['next_run']}")

                return self.job['next_run']

            def run(self, *args, **kwargs):
                # Wait until jstartdate occurs
                now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                if now < self.job['startdate']:
                    wait_time = (self.job['startdate'] - now).total_seconds()
                    print(f'Waiting for {wait_time} seconds to')
                    time.sleep(wait_time)

                # Check if end date is today and at time is in the past
                if self.job['end_date'] and self.job['end_date'].date() == now.date() and 'at' in self.job and self.job['at']:
                    hour, minute = map(int, self.job['at'].split(':'))
                    at_time = now.replace(hour=hour, minute=minute, second=0, microsecond=0)
                    if at_time < now.replace(second=0, microsecond=0):
                        raise ValueError("The specified hour is earlier than the current time for today's end date.")

                while True:
                    # defining now 1st time so we can run the function at the very first place
                    now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                    # print(f'THIS IS NOW {now}\n')
                    
                    # ensuring that code runs for the first time or at the time that it has to run next
                    if self.job['next_run'] is None or now >= self.job['next_run']:
                        # self.calculate_next_run(now)
                        print(f'next run is none or now something ')
                        # if self.job['next_run'].strftime('%Y-%m-%d %H:%M') == self.job['end date'].strftime('%Y-%m-%d %H:%M'):
                        #     print('in first break')
                        #     break
                        
                        # ensuring there is function to run 
                        if self.job['func']:
                            print(f'in function execution')
                            
                            # executing the function itself
                            self.job['func'](*self.job['args'], **self.job['kwargs'])
                            self.job['repeat_count'] += 1
                            
                            # overwriting on now variable so we can define it as the time when function ends running 
                            now = datetime.now(self.job['time_zone']).replace(microsecond=0)
                            
                            # if code reaches it's upper limit time it stops running 
                            if self.job['end_date'] and now >= self.job['end_date']:
                                print(f"endd{self.job['end_date']} , now {now}")
                                print('in mid break')
                                break
                            
                            print(f"This is overwritten now {now}")
                            # calculating nex time run after function gets executed
                            self.calculate_next_run(now)
                            print(f'this is repeat count: {self.job["repeat_count"]}, next run: {self.job["next_run"]}')
                            if self.job['repeats'] is not None and self.job['repeat_count'] >= self.job['repeats']:
                                print('repeatbreak?')
                                break
                        
                    if self.job['end_date'] and now >= self.job['end_date']:
                        print(f"endd{self.job['end_date']} , now {now}")
                        print('in second break')
                        break


        job_wrapper = JobWrapper(job)
        self.jobs.append(job_wrapper)
        return job_wrapper

    def run_all(self):
        if self.threads:
            threads = []
            for job in self.jobs:
                print(f"Starting thread for {job.job['name']}")
                thread = threading.Thread(target=job.run)
                threads.append(thread)
                thread.start()
            for thread in threads:
                thread.join()
        else:
            for job in self.jobs:
                job.run()




In [3]:


# Example functions
def report1(num1=1,num2 = 2):
    print(f"this is starting time {datetime.now()}")
    print(num1+num2)
    # time.sleep(10)
    print(f"finished running at {datetime.now()}\n")


# def report2():
#     print("Function report2 is running.")
# 
# def report3():
#     print("Function report3 is running.")

In [ ]:
scheduler = Scheduler(threading=True, time_zone=5)

# scheduler.job(time_zone=4).do(report1, 'report1').day(every=1, hour='15:38').repeat(1000).until('2024-09-02 15:40')
scheduler.job(time_zone=4).do(report1, 'report2').week(every=0.5).repeat(1000).until('2024-09-17 19:18')

scheduler.run_all()


inrepeat for report2
inuntil
Starting thread for report2
next run is none or now something 
in function execution
this is starting time 2024-09-17 19:16:57.675147
3
finished running at 2024-09-17 19:16:57.675147

This is overwritten now 2024-09-17 19:16:57+04:00
THISISWEEK2024-09-21 07:16:00+04:00
calc next run: 2024-09-21 07:16:00+04:00
this is repeat count: 1, next run: 2024-09-21 07:16:00+04:00


In [ ]:
    # def week(self, every=1, week_day=None, hour=None):
    #     """
    #     Set the job to run every `every` weeks on specific weekdays at a specific time.
    #     If `week_day` is None, the job will every week. 
    # 
    #     Parameters:
    #     -----------
    #     every : int
    #         Interval in weeks.
    #     week_day : int, list of int, or None
    #         Single integer or list of weekdays where 0 = Monday, ..., 6 = Sunday. If None, run every day.
    #     hour : str
    #         Time in HH:MM format at which the job should run.
    #     """
    #     self.job['unit'] = 'week'
    #     self.job['interval'] = every
    # 
    #     # Handling different types of inputs for week_day
    #     self.job['week_day'] = [week_day] if isinstance(week_day, int) else week_day if isinstance(week_day, list) else list(range(7))
    # 
    #     self.job['at'] = hour
    # 
    #     if hour:
    #         # Parse the time in HH:MM format
    #         hour, minute = map(int, hour.split(':'))
    #         today = datetime.now(self.job['time_zone']).replace(hour=hour, minute=minute, second=0, microsecond=0)
    #         next_run = None
    # 
    #         # Find the next valid run time based on the provided weekdays
    #         for day in sorted(self.job['week_day']):
    #             target_day = today + timedelta((day - today.weekday()) % 7)
    #             if target_day > today:
    #                 next_run = target_day
    #                 break
    # 
    #         # If no future days in the current week, move to the first valid day in the next interval week
    #         if not next_run:
    #             next_run = today + timedelta((self.job['week_day'][0] - today.weekday()) % 7 + 7 * (every - 1))
    # 
    #         # Check if the next run time is beyond the `until` time, raise an error if true
    #         if self.job['end_date'] and next_run > self.job['end_date']:
    #             raise ValueError(f"Unsupported day of the week: The next run time {next_run} exceeds the until time {self.job['end_date']}.")
    # 
    #         self.job['next_run'] = next_run
    #     return self

In [ ]:
# elif self.job['unit'] == 'week':
# # Set up the initial next run time based on the current time and interval
# next_run = current_time
# interval_weeks = self.job['interval']
# weekdays = self.job['week_day'] if self.job['week_day'] is not None else list(range(7))  # Default to all days of the week
# 
# # Find the nearest valid weekday that matches the job's criteria
# days_ahead = [(day - next_run.weekday()) % 7 for day in weekdays]
# days_ahead = [days_ahead if day != 0 else 7 for day in days_ahead]  # Adjust days_ahead to the nearest future valid day
# 
# # If we have a specific 'hour' to run the job, handle it
# if self.job.get('at'):
#     hour, minute = map(int, self.job['at'].split(':'))
#     next_run = next_run.replace(hour=hour, minute=minute, second=0, microsecond=0)
# 
#     # If the next run time has already passed today, set it for the next valid weekday
#     if next_run <= current_time:
#         next_run += timedelta(days=min(days_ahead))
# else:
#     next_run += timedelta(days=min(days_ahead))
# 
# # If the next run is still in the past, adjust by adding the weekly interval
# if next_run <= current_time:
#     next_run += timedelta(weeks=interval_weeks)
# 
# self.job['next_run'] = next_run
# else:
# raise ValueError("Unsupported unit. Please extend the `calculate_next_run` method to support other units.")
# 
# return self.job['next_run']